In [174]:
import re
import pickle
import Stemmer
import numpy as np
from collections import defaultdict, OrderedDict
import sys

In [175]:
re.split(':','b:Sachin c:world cup')

['b', 'Sachin c', 'world cup']

In [176]:
STOP_DICT = {}
# INPUT_FILE = '../enwiki-20200801-pages-articles-multistream1.xml-p1p30303'
INDEX_DIR = 'index/'
QUERY = 'Cristiano Ronaldo'
if INDEX_DIR[-1]!='/':
    INDEX_DIR+='/'
STOP_DICT = {}
STOP_FILE = ''
if INDEX_DIR.split('/')[0] == '2018114017':
    STOP_FILE = '2018114017/stopwords.pickle'
else:
    STOP_FILE = 'stopwords.pickle'
with open(STOP_FILE, 'rb') as handle:
    STOP_DICT = pickle.load(handle)
handle.close()
stemmer = Stemmer.Stemmer('english')
f = open(INDEX_DIR+"index.txt", "r")
index_string = f.read()
f.close()
temp = index_string.split('\n')
index = defaultdict(list)
for i in temp:
    splits = i.split(':')
    index[splits[0]] = splits[1].split(' ')

In [177]:
def preprocess(text):
    text = text.lower()
    tokens = re.sub(r'[^A-Za-z0-9]+', r' ', text).split()
    stemmed_stop_free = []
    for token in tokens:
        if token not in STOP_DICT:
            stemmed_stop_free.append(stemmer.stemWord(token))
    return stemmed_stop_free

In [178]:
def parse_query(query):
    query_list = query.split(':')
    if len(query_list) == 1:
        return preprocess(query_list[0]) , 0
    else:
        query_dict = {}
        # query_dict[query_list[0]] = ''
        for i in range(1, len(query_list)-1):
            query_dict[query_list[i-1][-1]] = preprocess(query_list[i][:-2])
        query_dict[query_list[-2][-1]] = preprocess(query_list[-1])
        return query_dict, 1

In [179]:
def run_whole_query(query):
    docs_intersect = []
    docs_postlist = {}
    for i in range(len(query)):
        doc_list = []
        post = []
        for j in index[query[i]]:
            splits = re.split('d|b|i|l|r|t|c', j)
            doc_list.append(splits[1])
            post.append(j)
        docs_postlist[query[i]]=post
        if i == 0:
            docs_intersect = doc_list
        else:
            docs_intersect = np.intersect1d(docs_intersect, doc_list)
    return docs_intersect, docs_postlist

In [180]:
def run_parsed_query(query_dict):
    flag = 0
    docs_intersect = []
    docs_postlist = {}
    for category in query_dict.keys():
        for i in range(len(query_dict[category])):
            doc_list = []
            post = []
            for j in index[query_dict[category][i]]:
                if category == 't':
                    splits = re.split('t', j)
                if category == 'b':
                    splits = re.split('b', j)
                if category == 'i':
                    splits = re.split('i', j)
                if category == 'c':
                    splits = re.split('c', j)
                if category == 'l':
                    splits = re.split('l', j)
                if category == 'r':
                    splits = re.split('r', j)
                if len(splits) > 1:
                    post.append(j)
                    splits2 = re.split('d|b|i|l|r|t|c', j)
                    doc_list.append(splits2[1])
                    docs_postlist[category+':'+query_dict[category][i]]=j
            docs_postlist[category+':'+query_dict[category][i]]=post
            if flag == 0:
                docs_intersect = doc_list
                flag = 1
            else:
                docs_intersect = np.intersect1d(docs_intersect, doc_list)
    return docs_intersect, docs_postlist

In [186]:
def print_postlist(postlist, docs_ids):
    for i in postlist.keys():
        print("Postlist for",i+":")
        for j in postlist[i]:
            print(j,end=' ')
        print()
        print()
    print('Common docs:', ' '.join(docs_ids))

In [187]:
parsed, querytype = parse_query(QUERY)
docs_intersect = []
docs_postlist = {}
if querytype == 0:
    docs_intersect, docs_postlist = run_whole_query(parsed)
else:
    docs_intersect, docs_postlist = run_parsed_query(parsed)
print_postlist(docs_postlist, docs_intersect)

Postlist for cristiano:
d943r1 d1493b1i1 d1870b1 d3389b1 d3454b3 d3526b1 d4152b1 d5235r1 d5318b1 d5342b1 d6285b1 d6694b4 d6989b1 d7066b4 d8497b1 d9244b1 d10088b1r1 d10095b1 d10184b1 d10483r1 d11926b1 d12897b5 d12939b1 d14597b1 d14983b4 d15568b1 d15606b1 d15788b2i2 d16344r1 d17270b9i1r4 d17357b2 d17435b1 d17968b1 d18049b1 d18393b1 d19181i2 d19335b1 

Postlist for ronaldo:
d3505b2 d5017r2 d5235b4r3 d5318b3 d6689b1 d6694b4 d6989b1 d7066b5r1 d7150b4 d9632b5r1 d9879b2 d10088r1 d10128b1 d10269b2 d10483r1 d10600b2 d12897b5 d12939b2 d14983b4 d16344b2r2 d16759r1 d16963b2r1 d17270b20i1r4 d18392b2 

Common docs: 10088 10483 12897 12939 14983 16344 17270 5235 5318 6694 6989 7066


In [183]:
# docs_postlist, parsed